# Web scrapping stock prices
This code accesses site to retrieve stock price information and then saves it to an individual csv file for each stock.

This code is for 10 YEAR STOCK HISTORY

The sources of data are:
- https://en.wikipedia.org/wiki/FTSE_100_Index -> list of FTSE 100 company stock tickers
- https://en.wikipedia.org/wiki/FTSE_250_Index -> list of FTSE 250 company stock tickers
- https://finance.yahoo.com/quote/{stock-ticker}/history?period1={start-time-mark}&period2={end-time-mark}&interval={interval}&filter=history&frequency={frequency} -> Example web address to retrieve information from Yahoo finance
    - Data on this page is scroll loaded so many time indexes must be used toretrieve the dcorrect data
    - Up to 145 records can be seen from initial page load note that this includes dividends so limit to 140 for safety

The inputs required for scrapping are:
 - {stock-ticker} -> this is the ticker taken from wiki with ".L" appended to it
 - {start-tme-mark} -> This is the time in seconds since 01/01/1970 at which you would like the data retrieval to start, data retrieved is inclusive of this time
 - {end-tme-mark} -> This is the time in seconds since 01/01/1970, data retrieved is inclusive of this time
 - {interval} & {frequency} -> This is the interval for which values are given, the two must match
     - 1d = 1 every 1 days
     - 1wk = 1 every week
     - 1mo = 1 eveery month

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests as rq
import re
import datetime as dt
import os
import tables

In [2]:
#Set options for matplotlib
%matplotlib inline

In [3]:
class process_time:
    def __init__(self,name:str = ''):
        self.st_time = dt.datetime.now()
        self.lap_li = []
        self.en_time = None
        self.name = name
    def calc_el_time(self,st_time,en_time):
        diff_time = en_time - st_time
        duration_in_s = diff_time.total_seconds()
        hours = int(divmod(duration_in_s, 3600)[0])
        duration_in_s += -(hours * 3600)
        minutes = int(divmod(duration_in_s, 60)[0])
        duration_in_s += -(minutes * 60)
        seconds = int(duration_in_s)
        return [hours,minutes,seconds]
    def lap(self):
        self.lap_li.append(dt.datetime.now())
    def end(self):
        self.en_time = dt.datetime.now()
        lap_time = self.calc_el_time(self.st_time,self.en_time)
        if self.name != '':
            print('TOTAL ELAPSED TIME OF {} -> {}:{}:{}'.format(self.name,lap_time[0],lap_time[1],lap_time[2]))
        else:
            print('TOTAL ELAPSED TIME -> {}:{}:{}'.format(lap_time[0],lap_time[1],lap_time[2]))
    def show_lap_times(self):
        tmp_count = 0
        for lap in self.lap_li:
            tmp_count += 1
            lap_time = self.calc_el_time(self.st_time,lap)
            print('LAP {} TIME -> {}:{}:{}'.format(tmp_count,lap_time[0],lap_time[1],lap_time[2]))
    def show_latest_lap_time(self):
        if len(self.lap_li) == 0:
            return
        elif len(self.lap_li) < 2:
            lap_time = self.calc_el_time(self.st_time,self.lap_li[-1])
        else:
            lap_time = self.calc_el_time(self.lap_li[-2],self.lap_li[-1])
        print('LAP {} TIME -> {}:{}:{}'.format(len(self.lap_li),lap_time[0],lap_time[1],lap_time[2]))

# Delete the old temporary files (if they exist)

In [4]:
#close any open h5 files
tables.file._open_files.close_all()

In [5]:
#Delete the old h5 files
src_fldr_pth = r'C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\'
try:
    os.remove(src_fldr_pth + r'all_hist_prices_d_TMP.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_d_TMP.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.remove(src_fldr_pth + r'all_hist_prices_w_TMP.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_w_TMP.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))


SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d_TMP.h5

SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w_TMP.h5


# Scraping tickers
This section will scrap the ticker values for the FTSE 100 and FTSE 250 and store them in dataframes "tick_ftse100" and "tick_ftse250".

Finally concatenate into 1 dataframe "tick_ftse".

In [6]:
#Fetch the data for ftse 100
web_add = 'https://en.wikipedia.org/wiki/FTSE_100_Index'
resp = rq.get(web_add)
parser = bs(resp.content,'html.parser')
#Get the table id=constituents
table = parser.find_all('table',id='constituents')[0]
#Get rows ignoring first row
rows = table.find_all('tr')[1:]
row_li = []
for r in rows:
    temp_row = []
    for r2 in r.find_all('td'):
        temp_row.append(re.sub('(?=[A-Z]*)\..*','',r2.text.upper().rstrip()))
    temp_row = temp_row[:2]
    row_li.append(temp_row)
#Create a dataframe
tick_ftse100 = pd.DataFrame(data=row_li,columns=['company','ticker'])
tick_ftse100['index'] = 'FTSE100'
tick_ftse100

,company,ticker,index
0,3I,III,FTSE100
1,ADMIRAL GROUP,ADM,FTSE100
2,ANGLO AMERICAN PLC,AAL,FTSE100
3,ANTOFAGASTA,ANTO,FTSE100
4,ASHTEAD GROUP,AHT,FTSE100
5,ASSOCIATED BRITISH FOODS,ABF,FTSE100
6,ASTRAZENECA,AZN,FTSE100
7,AUTO TRADER GROUP,AUTO,FTSE100
8,AVEVA,AVV,FTSE100
9,AVIVA,AV,FTSE100


In [7]:
#Fetch the data for ftse 250
web_add = 'https://en.wikipedia.org/wiki/FTSE_250_Index'
resp = rq.get(web_add)
parser = bs(resp.content,'html.parser')
#Get the table id=constituents
table = parser.find_all('table',id='constituents')[0]
#Get rows ignoring first row
rows = table.find_all('tr')[1:]
row_li = []
for r in rows:
    temp_row = []
    for r2 in r.find_all('td'):
        temp_row.append(re.sub('(?=[A-Z]*)\..*','',r2.text.upper().rstrip()))
    temp_row = temp_row[:2]
    row_li.append(temp_row)
#Create a dataframe
tick_ftse250 = pd.DataFrame(data=row_li,columns=['company','ticker'])
tick_ftse250['index'] = 'FTSE250'
tick_ftse250

,company,ticker,index
0,3I INFRASTRUCTURE,3IN,FTSE250
1,4IMPRINT,FOUR,FTSE250
2,ABERFORTH SMALLER COMPANIES TRUST,ASL,FTSE250
3,ACACIA MINING,ACA,FTSE250
4,AGGREKO,AGK,FTSE250
5,ALLIANCE TRUST,ATST,FTSE250
6,AMIGO,AMGO,FTSE250
7,APAX GLOBAL ALPHA,APAX,FTSE250
8,ASCENTIAL,ASCL,FTSE250
9,ASHMORE GROUP,ASHM,FTSE250


In [8]:
#Combine into 1 dataframe
tick_ftse = pd.concat([tick_ftse100,tick_ftse250])
tick_ftse

,company,ticker,index
0,3I,III,FTSE100
1,ADMIRAL GROUP,ADM,FTSE100
2,ANGLO AMERICAN PLC,AAL,FTSE100
3,ANTOFAGASTA,ANTO,FTSE100
4,ASHTEAD GROUP,AHT,FTSE100
5,ASSOCIATED BRITISH FOODS,ABF,FTSE100
6,ASTRAZENECA,AZN,FTSE100
7,AUTO TRADER GROUP,AUTO,FTSE100
8,AVEVA,AVV,FTSE100
9,AVIVA,AV,FTSE100


# Scraping a stock history
Working backwards through time from now until 01/01/1970 collect all the daily data for a stock. Daily is used as it can later be summarised into weekly or monthly if required.

This will then be put into a dataframe containing:
- Ticker
- Company name
- Date
- Open
- High
- Low
- Close
- Adjusted close (for divs and splits)
- Volume

In [9]:
#Create a list of time intervals to be used with 140 days in each item
def create_sec_ref_li(_st_date:int,_en_date:int):
    #Establish the day ref of the dates compared to 01/01/1970
    _ep_date = pd.to_datetime(dt.datetime(1970,1,1),errors='coerce')
    _en_date = pd.to_datetime(_en_date,errors='coerce')
    _st_date = pd.to_datetime(_st_date,errors='coerce')
    print('_st_date: ' + str(_st_date))
    print('_en_date: ' + str(_en_date))
    _st_days = (_st_date - _ep_date).days
    _en_days = (_en_date - _ep_date).days
    #Loop adding to a list until reaching 0
    _sec_ref_li = []
    _days = 140
    while _en_days > _st_days:
        if _en_days - _days > _st_days:
            _sec_ref_li.append([(_en_days - _days)*86400,_en_days*86400])
        else:
            _sec_ref_li.append([_st_days*86400,_en_days*86400])        
        _en_days += -_days
    return _sec_ref_li

In [11]:
def calc_ema(s_in,periods):
    #Calc mod val
    mod = 2/(periods+1)
    #Make a df
    tmp_df = pd.DataFrame(s_in)
    #Calc sma
    tmp_df["sma"] = pd.Series([0] * len(s_in))
    for i in range(0,periods):
        tmp_df["sma"] += tmp_df[s_in.name].shift(i) / periods
    #Calc ema
    tmp_df["ema"] = tmp_df["sma"].copy()
    for row in tmp_df.iterrows():
        i = row[0]
        if i > 0 and not np.isnan(tmp_df["sma"][i-1]):
            tmp_df["ema"][i] = mod*(tmp_df[s_in.name][i] - tmp_df["ema"][i-1]) + tmp_df["ema"][i-1]
    return tmp_df["ema"].copy()

In [12]:
def calc_macd(ema_lng_s,ema_sht_s,sig_period):
    #Make a df
    tmp_df = pd.DataFrame([])
    tmp_df["ema_lng"] = ema_lng_s
    tmp_df["ema_sht"] = ema_sht_s
    #Calc the signal line
    tmp_df["macd_line"] = tmp_df["ema_sht"] - tmp_df["ema_lng"]
    tmp_df["signal"] = calc_ema(tmp_df["macd_line"],sig_period)
    tmp_df["macd_hist"] = tmp_df["macd_line"] - tmp_df["signal"]
    return (tmp_df["macd_line"].copy(),tmp_df["signal"].copy(),tmp_df["macd_hist"].copy())

In [13]:
col_lens = {
    'ticker': 4,
     'date': 19,
     'open': 7,
     'close': 7,
     'high': 7,
     'low': 7,
     'change': 21,
     'volume': 12,
     'ema12': 18,
     'ema26': 18,
     'macd_line': 23,
     'signal': 23,
     'macd': 23}

SyntaxError: invalid syntax (<ipython-input-13-21ec72687d7c>, line 1)

In [ ]:
#Get the price history for a specific ticker
def get_price_hist_d(_tick:str,_sec_ref_li:list):
    try:
        _tick = re.sub('[^A-Z0-9\-]','',_tick)
        print('Getting DAILY prices for:{}'.format(_tick))
        _tick_df = pd.DataFrame([])

        for _secs in _sec_ref_li:
            try:
                _web_add = 'https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval={3}&filter=history&frequency={3}'.format(_tick+'.L',_secs[0],_secs[1],'1d')
                print('_web_add: {}'.format(_web_add))
                _resp = rq.get(_web_add)
                print('status code: {}'.format(_resp.status_code))
                _parser = bs(_resp.content,'html.parser')
                #Get the table
                _table = _parser.find_all('table',attrs={'data-test':'historical-prices'})[0]
                #Grab the data rows
                _rows = _table.find_all('tbody')[0].find_all('tr')
                #Put the rows into the dataframe
                _cols = []
                for _r in _rows:
                    if len(_tick_df) == 0:
                        _cols = [clean_col_name(x.text) for x in _table.find_all('th')]
                        _tick_df = pd.DataFrame([],columns=_cols)
                    if len(_r.find_all('td')) == len(_cols):
                        _tick_df = _tick_df.append(pd.Series([x.text for x in _r.find_all('td')],index=_cols),ignore_index=True)
                    else:
                        continue
            except Exception as e:
                print('ERROR - CONTINUE:{}'.format(e))
                continue
        #Check for rows - if none then return
        if len(_tick_df) == 0:
            return _tick_df
        #Reformat
        def float_format(_str_in):
            if type(_str_in) == str:
                _str_in = _str_in.strip()
                _str_in = re.sub('[^0-9.]','',_str_in)
                if _str_in == '':
                    _str_in = 0
                return _str_in
            else:
                return _str_in
        _tick_df.loc[:,'open'] = _tick_df.loc[:,'open'].apply(float_format).astype(float)
        _tick_df.loc[:,'high'] = _tick_df.loc[:,'high'].apply(float_format).astype(float)
        _tick_df.loc[:,'low'] = _tick_df.loc[:,'low'].apply(float_format).astype(float)
        _tick_df.loc[:,'close'] = _tick_df.loc[:,'close*'].apply(float_format).astype(float)
        _tick_df.loc[:,'adj_close'] = _tick_df.loc[:,'adj_close'].apply(float_format).astype(float)
        _tick_df.loc[:,'volume'] = _tick_df.loc[:,'volume'].apply(float_format).astype(float)
        _tick_df.loc[:,'change'] = _tick_df.loc[:,'close'] - _tick_df.loc[:,'open']
        def conv_date(_str_in):
            if type(_str_in) == str:
                return dt.datetime.strptime(_str_in,'%b %d, %Y')
            else:
                return _str_in
        _tick_df.loc[:,'date'] = _tick_df.loc[:,'date'].apply(conv_date)
        #Add the ticker series
        _tick_df.loc[:,'ticker'] = _tick    
        _tick_df = _tick_df.loc[:,['ticker','date','open','close','high','low','change','volume']]
        #CLEANING - Remove any rows with zero volume
        _tick_df = _tick_df[_tick_df['volume'] > 0]
        #CLEANING - Copy row above where the change has been more than 90%
        _tick_df['cl_change'] = (_tick_df['close'] - _tick_df['close'].shift(1))/_tick_df['close'].shift(1)
        _check_s = _tick_df['cl_change'] < -0.9
        _tick_df.loc[_check_s,'open'] = _tick_df['open'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'close'] = _tick_df['close'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'high'] = _tick_df['high'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'low'] = _tick_df['low'].shift(-1).copy().loc[_check_s]
        _tick_df.drop(columns='cl_change',inplace=True)
        return _tick_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

#Calc the ema and macds for the data
def calc_ema_macd(_tick_df):
    try:
        #Add in the ema and macd
        _tick_df = _tick_df.sort_values(by='date')
        _tick_df = _tick_df.reset_index(drop=True)
        _tick_df['ema12'] = calc_ema(_tick_df['close'],12)
        _tick_df['ema26'] = calc_ema(_tick_df['close'],26)
        _tick_df['macd_line'],_tick_df['signal'],_tick_df['macd'] = calc_macd(_tick_df['ema26'],_tick_df['ema12'],9)
        #Sort clean and export
        _tick_df = _tick_df.sort_values(by='date')
        _tick_df = _tick_df.reset_index(drop=True)
        return _tick_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

In [ ]:
#Create a weekly table
def get_price_hist_w(_df_d):
    print('Converting daily prices to weekly prices')
    try:
        #Create a copy of the data
        _df_d = _df_d.copy()
        #Establish a week number for each date
        _df_d['isocalendar'] = [x.isocalendar()[:2] for x in _df_d['date']]
        #Get highs and lows
        _high_df = _df_d[['high','isocalendar']].groupby('isocalendar').max().reset_index()
        _low_df = _df_d[['low','isocalendar']].groupby('isocalendar').min().reset_index()
        #Get total volume for the week
        _vol_df = _df_d[['volume','isocalendar']].groupby('isocalendar').sum().reset_index()
        #Get open price
        _min_wk_day = _df_d[['date','isocalendar']].groupby('isocalendar').min().reset_index()
        _open_df = pd.merge(_df_d[['date','open']],_min_wk_day,left_on='date',right_on='date')
        #Get close price
        _max_wk_day = _df_d[['date','isocalendar']].groupby('isocalendar').max().reset_index()
        _close_df = pd.merge(_df_d[['date','close']],_max_wk_day,left_on='date',right_on='date').reset_index()
        #Form the final df
        _wk_df = pd.merge(_df_d[['ticker','isocalendar']],_min_wk_day,left_on='isocalendar',right_on='isocalendar') #date
        _wk_df = pd.merge(_wk_df,_high_df,left_on='isocalendar',right_on='isocalendar') #high
        _wk_df = pd.merge(_wk_df,_low_df,left_on='isocalendar',right_on='isocalendar') #low
        _wk_df = pd.merge(_wk_df,_vol_df,left_on='isocalendar',right_on='isocalendar') #volume
        _wk_df = pd.merge(_wk_df,_open_df[['isocalendar','open']],left_on='isocalendar',right_on='isocalendar') #open
        _wk_df = pd.merge(_wk_df,_close_df[['isocalendar','close']],left_on='isocalendar',right_on='isocalendar') #close
        _wk_df['change'] = _wk_df['close'] - _wk_df['open']
        _wk_df = _wk_df.drop_duplicates().reset_index(drop=True)
        #Get the monday of each week
        _wk_df['weekday'] = [dt.date.weekday(x) for x in _wk_df['date']]
        _wk_df['date'] = _wk_df['date'] - pd.Series([dt.timedelta(days=x) for x in _wk_df['weekday']])
        _wk_df.drop(columns=['isocalendar','weekday'],inplace=True)
        return _wk_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

In [ ]:
#Scrape daily price data
hf_store_name = src_fldr_pth + r'all_hist_prices_d_TMP.h5'
hf = pd.HDFStore(hf_store_name)
group_name = r'daily_data'
out_cols = ['ticker','date','open','close','high','low','change','volume']
count = 0
errors = []
run_time = process_time()
for tick in tick_ftse['ticker']:
    try:
        run_time.show_latest_lap_time()
        run_time.lap()
        count += 1
        print('\n{} RUNNING FOR: {}'.format(count,tick))
        tick_df = hist_prices_df[hist_prices_df.ticker == tick][out_cols]
        print('SHAPE BEFORE: {}'.format(tick_df.shape))

        #Get last scrapped date
        st_date = tick_df.date.max()
        if pd.isnull(st_date): 
            #Treat as a new share
            st_date = dt.datetime(1970,1,1)

        #Get new price data
        new_tick_df = get_price_hist_d(tick,create_sec_ref_li(st_date,dt.datetime.today()+dt.timedelta(days=1))) #1 day ahead so today is included

        #Join onto existing data
        tick_df = tick_df.append(new_tick_df)

        #Calc emas and macd
        tick_df = calc_ema_macd(tick_df)
        print('SHAPE AFTER: {}'.format(tick_df.shape))

        #Clarify col_lens with cur cols in data
        col_lens_tmp = {}
        for col in tick_df:
            if col in col_lens:
                col_lens_tmp[col] = col_lens[col]

        #Add to h5 file
        tick_df.to_hdf(hf_store_name,key=group_name,append=True,min_itemsize=col_lens_tmp)
    except Exception as e:
        print('ERROR:{}'.format(e))
        errors.append(e)
hf.close()
print('\n\n')
run_time.end()
print('\nERROR COUNT: {}'.format(len(errors)))
if len(errors) > 0:
    print('    ERRORS -> {}'.format(errors))

In [ ]:
def show_time_el(st_time,en_time):
    diff_time = en_time - st_time
    duration_in_s = diff_time.total_seconds()
    hours = int(divmod(duration_in_s, 3600)[0])
    duration_in_s += -(hours * 3600)
    minutes = int(divmod(duration_in_s, 60)[0])
    duration_in_s += -(minutes * 60)
    seconds = int(duration_in_s)
    print("TIME ELAPSED - {}:{}:{}".format(hours,minutes,seconds))

In [ ]:
#Run all tickers and export
hf_store_name = src_fldr_pth + r'all_hist_prices_w_TMP.h5'
hf = pd.HDFStore(hf_store_name)
group_name = r'weekly_data'
out_cols = ['ticker','date','open','close','high','low','change','volume']
print('TOTAL TO RUN: {}'.format(len(tick_ftse['ticker'])))
count = 0
errors = []
run_time = process_time()
print("TOTAL TO RUN: {}".format(len(tick_ftse["Ticker"])))
count = 0
for tick in tick_ftse["Ticker"]:
    st_time = dt.datetime.now()
    print("")
    st_time = 
    count += 1
    print(count)
    print("RUNNING FOR:{}".format(tick))
    temp_df_d = get_price_hist_d(tick)
    temp_df_w = get_price_hist_w(temp_df_d)
    exp_csv(tick,temp_df_d,temp_df_w)
    en_time = dt.datetime.now()
    #Show time elapsed
    show_time_el(st_time,en_time)

In [ ]:
#Export the ftse list
path = "C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\"
tick_ftse.to_csv(path_or_buf=path + "tick_ftse.csv")